# <span style="font-size: 28px;"><b>Demo: Integrate Statistical Methods for Improved Model Performance</b></span>

## **Step 1: Extract Data from SQL Server**

In [30]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import warnings
warnings.filterwarnings("ignore")

# Connection string components
server = 'DESKTOP'               # Server name
database = 'Transactions'        # Database name
driver= 'SQL Server'                   

# SQLAlchemy connection string
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}&trusted_connection=yes'

# Create the engine
engine = create_engine(connection_string)

query = '''
SELECT transaction_id, customer_id, 
       CAST(date AS DATE) as date, 
       CAST(time AS TIME) as time, 
       product_name, category, quantity, price
FROM dbo.Transactions
'''

# Use the engine to connect and execute the query
df = pd.read_sql_query(query, engine)

## **Step 2: Feature Engineering**

In [31]:
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df['month'] = df['datetime'].dt.month
df['day_of_week'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

## **Step 3:** **Cyclical Encoding**

In [32]:
# Cyclical encoding for hour
df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)

## **Step 4:** **Feature Preprocessing**

In [33]:
# Dropping the original date, time, and datetime columns
df.drop(['date', 'time', 'datetime', 'hour'], axis=1, inplace=True)

# Target and features
X = df.drop(['transaction_id', 'price'], axis=1)
y = df['price']

# Encoding categorical variables and scaling numerical variables
categorical_features = ['customer_id', 'product_name', 'category']
numerical_features = ['quantity', 'month', 'day_of_week', 'hour_sin', 'hour_cos']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ])

## **Step 5:** **Initialize Model**

In [34]:
# Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=44)

# Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=44))),
    ('regressor', model)
])

## **Step 6:** **Splitting Dataset**

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

## **Step 7:** **Model** **Fitting**

In [36]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['quantity', 'month',
                                                   'day_of_week', 'hour_sin',
                                                   'hour_cos']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['customer_id',
                                                   'product_name',
                                                   'category'])])),
                ('feature_selection',
                 SelectFromModel(estimator=RandomForestRegressor(random_state=44))),
                ('regressor', RandomForestRegressor(random_state=44))])

## **Step 8:** **Model Predictions**

In [37]:
predictions = pipeline.predict(X_test)

## **Step 9:** **Evaluation**

In [38]:
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Absolute Error: 12.581256776759963
Mean Squared Error: 274.9406898836552
R^2 Score: 0.9960512917323361
